# IRNet training

# 1. SSUB

In [1]:
#https://github.com/wolverton-research-group/qmpy/blob/master/qmpy/data/thermodata/ssub.dat
from jarvis.db.figshare import  get_request_data
dat = get_request_data(js_tag="ssub.json",url="https://figshare.com/ndownloader/files/40084921")


Loading the zipfile...
Loading completed.


0

In [2]:
import pandas as pd
df = pd.DataFrame(dat)

In [3]:
df

,id,formula_energy,formula
0,465,-2.841796e+00,CaO4W
1,1197,1.766051e-09,Ho
2,1759,-3.555857e-01,PbTe
3,1454,-9.540103e-01,MoS2
4,218,-6.564038e-02,BeH2
...,...,...,...
1722,1482,-1.892842e+00,NZr
1723,773,-2.404511e+00,CrLiO2
1724,251,-4.946426e-01,BiNa3
1725,328,-7.213532e-01,Br4Ge


In [4]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/ssub_formula_energy.json.zip"
js_tag = "ssub_formula_energy.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))

In [5]:
id_data.keys()

dict_keys(['train', 'test'])

In [6]:
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')

In [7]:
print(len(train_ids),len(test_ids),len(train_ids)+len(test_ids))

1381 345 1726


In [8]:
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [9]:
len(train_df),  len(test_df)

(1381, 345)

In [10]:
%%time
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['formula_energy'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['formula_energy'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')


CPU times: user 261 ms, sys: 32.5 ms, total: 294 ms
Wall time: 404 ms


In [11]:
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [12]:
X_train.shape,X_test.shape

((1104, 103), (345, 103))

In [15]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [16]:
#Network from "Enabling deeper learning on big data for materials informatics applications"
#Link to paper: https://dl.acm.org/doi/10.1145/3292500.3330703 https://www.nature.com/articles/s41598-021-83193-1
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = BatchNormalization()(layer_1)
layer_1 = Activation('relu')(layer_1)

fcc_1 = Dense(1024)(in_layer)
gsk_1 = add([fcc_1, layer_1])

layer_2 = Dense(1024)(gsk_1)
layer_2 = BatchNormalization()(layer_2)
layer_2 = Activation('relu')(layer_2)

gsk_2 = add([gsk_1, layer_2])

layer_3 = Dense(1024)(gsk_2)
layer_3 = BatchNormalization()(layer_3)
layer_3 = Activation('relu')(layer_3)

gsk_3 = add([gsk_2, layer_3])

layer_4 = Dense(1024)(gsk_3)
layer_4 = BatchNormalization()(layer_4)
layer_4 = Activation('relu')(layer_4)

gsk_4 = add([gsk_3, layer_4])

layer_5 = Dense(512)(gsk_4)
layer_5 = BatchNormalization()(layer_5)
layer_5 = Activation('relu')(layer_5)

mcc_5 = Dense(512)(gsk_4)
msk_5 = add([mcc_5, layer_5])

layer_6 = Dense(512)(msk_5)
layer_6 = BatchNormalization()(layer_6)
layer_6 = Activation('relu')(layer_6)

msk_6 = add([msk_5, layer_6])

layer_7 = Dense(512)(msk_6)
layer_7 = BatchNormalization()(layer_7)
layer_7 = Activation('relu')(layer_7)

msk_7 = add([msk_6, layer_7])

layer_8 = Dense(256)(msk_7)
layer_8 = BatchNormalization()(layer_8)
layer_8 = Activation('relu')(layer_8)

mcc_8 = Dense(256)(msk_7)
msk_8 = add([mcc_8, layer_8])

layer_9 = Dense(256)(msk_8)
layer_9 = BatchNormalization()(layer_9)
layer_9 = Activation('relu')(layer_9)

msk_9 = add([msk_8, layer_9])

layer_10 = Dense(256)(msk_9)
layer_10 = BatchNormalization()(layer_10)
layer_10 = Activation('relu')(layer_10)

msk_10 = add([msk_9, layer_10])

layer_11 = Dense(128)(layer_10)
layer_11 = BatchNormalization()(layer_11)
layer_11 = Activation('relu')(layer_11)

mcc_11 = Dense(128)(msk_10)
msk_11 = add([mcc_11, layer_11])

layer_12 = Dense(128)(msk_11)
layer_12 = BatchNormalization()(layer_12)
layer_12 = Activation('relu')(layer_12)

msk_12 = add([msk_11, layer_12])

layer_13 = Dense(128)(msk_12)
layer_13 = BatchNormalization()(layer_13)
layer_13 = Activation('relu')(layer_13)

msk_13 = add([msk_12, layer_13])

layer_14 = Dense(64)(msk_13)
layer_14 = BatchNormalization()(layer_14)
layer_14 = Activation('relu')(layer_14)

mcc_14 = Dense(64)(msk_13)
msk_14 = add([mcc_14, layer_14])

layer_15 = Dense(64)(msk_14)
layer_15 = BatchNormalization()(layer_15)
layer_15 = Activation('relu')(layer_15)

msk_15 = add([msk_14, layer_15])

layer_16 = Dense(32)(msk_15)
layer_16 = BatchNormalization()(layer_16)
layer_16 = Activation('relu')(layer_16)

mcc_16 = Dense(32)(msk_15)
msk_16 = add([mcc_16, layer_16])

out_layer = Dense(1)(msk_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [17]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000


2023-05-22 18:46:08.237492: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f68e4003b80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-22 18:46:08.237568: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla V100-PCIE-16GB, Compute Capability 7.0
2023-05-22 18:46:08.237594: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (1): Tesla V100-PCIE-16GB, Compute Capability 7.0
2023-05-22 18:46:08.252966: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-22 18:46:08.755380: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


35/35 - 16s - loss: 8.1191 - mean_absolute_error: 8.1191 - val_loss: 0.9005 - val_mean_absolute_error: 0.9005 - 16s/epoch - 443ms/step
Epoch 2/3000
35/35 - 1s - loss: 0.9846 - mean_absolute_error: 0.9846 - val_loss: 0.8483 - val_mean_absolute_error: 0.8483 - 945ms/epoch - 27ms/step
Epoch 3/3000
35/35 - 1s - loss: 0.8023 - mean_absolute_error: 0.8023 - val_loss: 0.9098 - val_mean_absolute_error: 0.9098 - 847ms/epoch - 24ms/step
Epoch 4/3000
35/35 - 1s - loss: 0.8557 - mean_absolute_error: 0.8557 - val_loss: 1.0927 - val_mean_absolute_error: 1.0927 - 870ms/epoch - 25ms/step
Epoch 5/3000
35/35 - 1s - loss: 0.7422 - mean_absolute_error: 0.7422 - val_loss: 1.3172 - val_mean_absolute_error: 1.3172 - 976ms/epoch - 28ms/step
Epoch 6/3000
35/35 - 1s - loss: 0.7025 - mean_absolute_error: 0.7025 - val_loss: 1.1743 - val_mean_absolute_error: 1.1743 - 919ms/epoch - 26ms/step
Epoch 7/3000
35/35 - 1s - loss: 0.6879 - mean_absolute_error: 0.6879 - val_loss: 0.9163 - val_mean_absolute_error: 0.9163 - 8

In [18]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

11/11 [==============================] - 0s 5ms/step
0.12876772967171163


In [19]:
# pred=model.predict(X_test)
# print(mean_absolute_error(y_test,pred))
# 0.17889024475861343

In [20]:
import os
f=open('AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred[0]):
    line=str(i)+','+str(j)+','+str(k)+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv.zip AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-formula_energy-ssub-test-mae.csv (deflated 4%)


0

# 2. SuperCon

In [21]:
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="supercon_chem.json",url="https://figshare.com/ndownloader/files/40719260")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)

Loading the zipfile...
Loading completed.


In [22]:
df

,id,Tc,formula
0,1524,0.00,Tb1Th3.4Ru0.6B4
1,14309,0.00,B2Th1
2,6607,1.09,Ni2.5Cu1.5Zr8
3,4010,24.60,Pr0.91La1Ce0.09Cu1O4
4,14511,55.00,Pb0.5Ca1Sr2Y0.5Cu2O6.97
...,...,...,...
16409,442,0.00,As1Pd7
16410,14942,0.00,La1.5Gd1.5Ca1Ba2Sr1Cu6.3In0.7O16.62
16411,3978,0.00,N0.98Pr1
16412,1078,32.00,Sm0.1La1.7Sr0.2Cu1O4


In [23]:
import json,zipfile
import numpy as np
path = "../../benchmarks/AI/SinglePropertyPrediction/supercon_chem_Tc.json.zip"
js_tag = "supercon_chem_Tc.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

In [24]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error


X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['Tc'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['Tc'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [25]:
X_train.shape,X_test.shape

((10504, 103), (3283, 103))

In [26]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error
#Network from "Enabling deeper learning on big data for materials informatics applications"
#Link to paper: https://dl.acm.org/doi/10.1145/3292500.3330703 https://www.nature.com/articles/s41598-021-83193-1
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = BatchNormalization()(layer_1)
layer_1 = Activation('relu')(layer_1)

fcc_1 = Dense(1024)(in_layer)
gsk_1 = add([fcc_1, layer_1])

layer_2 = Dense(1024)(gsk_1)
layer_2 = BatchNormalization()(layer_2)
layer_2 = Activation('relu')(layer_2)

gsk_2 = add([gsk_1, layer_2])

layer_3 = Dense(1024)(gsk_2)
layer_3 = BatchNormalization()(layer_3)
layer_3 = Activation('relu')(layer_3)

gsk_3 = add([gsk_2, layer_3])

layer_4 = Dense(1024)(gsk_3)
layer_4 = BatchNormalization()(layer_4)
layer_4 = Activation('relu')(layer_4)

gsk_4 = add([gsk_3, layer_4])

layer_5 = Dense(512)(gsk_4)
layer_5 = BatchNormalization()(layer_5)
layer_5 = Activation('relu')(layer_5)

mcc_5 = Dense(512)(gsk_4)
msk_5 = add([mcc_5, layer_5])

layer_6 = Dense(512)(msk_5)
layer_6 = BatchNormalization()(layer_6)
layer_6 = Activation('relu')(layer_6)

msk_6 = add([msk_5, layer_6])

layer_7 = Dense(512)(msk_6)
layer_7 = BatchNormalization()(layer_7)
layer_7 = Activation('relu')(layer_7)

msk_7 = add([msk_6, layer_7])

layer_8 = Dense(256)(msk_7)
layer_8 = BatchNormalization()(layer_8)
layer_8 = Activation('relu')(layer_8)

mcc_8 = Dense(256)(msk_7)
msk_8 = add([mcc_8, layer_8])

layer_9 = Dense(256)(msk_8)
layer_9 = BatchNormalization()(layer_9)
layer_9 = Activation('relu')(layer_9)

msk_9 = add([msk_8, layer_9])

layer_10 = Dense(256)(msk_9)
layer_10 = BatchNormalization()(layer_10)
layer_10 = Activation('relu')(layer_10)

msk_10 = add([msk_9, layer_10])

layer_11 = Dense(128)(layer_10)
layer_11 = BatchNormalization()(layer_11)
layer_11 = Activation('relu')(layer_11)

mcc_11 = Dense(128)(msk_10)
msk_11 = add([mcc_11, layer_11])

layer_12 = Dense(128)(msk_11)
layer_12 = BatchNormalization()(layer_12)
layer_12 = Activation('relu')(layer_12)

msk_12 = add([msk_11, layer_12])

layer_13 = Dense(128)(msk_12)
layer_13 = BatchNormalization()(layer_13)
layer_13 = Activation('relu')(layer_13)

msk_13 = add([msk_12, layer_13])

layer_14 = Dense(64)(msk_13)
layer_14 = BatchNormalization()(layer_14)
layer_14 = Activation('relu')(layer_14)

mcc_14 = Dense(64)(msk_13)
msk_14 = add([mcc_14, layer_14])

layer_15 = Dense(64)(msk_14)
layer_15 = BatchNormalization()(layer_15)
layer_15 = Activation('relu')(layer_15)

msk_15 = add([msk_14, layer_15])

layer_16 = Dense(32)(msk_15)
layer_16 = BatchNormalization()(layer_16)
layer_16 = Activation('relu')(layer_16)

mcc_16 = Dense(32)(msk_15)
msk_16 = add([mcc_16, layer_16])

out_layer = Dense(1)(msk_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [27]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
329/329 - 20s - loss: 12.4074 - mean_absolute_error: 12.4074 - val_loss: 22.7054 - val_mean_absolute_error: 22.7054 - 20s/epoch - 60ms/step
Epoch 2/3000
329/329 - 8s - loss: 10.7250 - mean_absolute_error: 10.7250 - val_loss: 9.5290 - val_mean_absolute_error: 9.5290 - 8s/epoch - 25ms/step
Epoch 3/3000
329/329 - 8s - loss: 10.3004 - mean_absolute_error: 10.3004 - val_loss: 9.7547 - val_mean_absolute_error: 9.7547 - 8s/epoch - 25ms/step
Epoch 4/3000
329/329 - 9s - loss: 9.9722 - mean_absolute_error: 9.9722 - val_loss: 8.8748 - val_mean_absolute_error: 8.8748 - 9s/epoch - 26ms/step
Epoch 5/3000
329/329 - 8s - loss: 10.0512 - mean_absolute_error: 10.0512 - val_loss: 9.2328 - val_mean_absolute_error: 9.2328 - 8s/epoch - 24ms/step
Epoch 6/3000
329/329 - 8s - loss: 9.7333 - mean_absolute_error: 9.7333 - val_loss: 9.6734 - val_mean_absolute_error: 9.6734 - 8s/epoch - 25ms/step
Epoch 7/3000
329/329 - 8s - loss: 9.4334 - mean_absolute_error: 9.4334 - val_loss: 17.7796 - val_mean_abso

In [28]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

103/103 [==============================] - 1s 6ms/step
5.834958187930562


In [29]:
import os
f=open('AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv.zip AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-Tc-supercon_chem-test-mae.csv (deflated 52%)


0

# 3. Magnetic2DChem

In [30]:
import json,zipfile
import numpy as np
from jarvis.db.figshare import  get_request_data
import pandas as pd
dat = get_request_data(js_tag="mag2d_chem.json",url="https://figshare.com/ndownloader/files/40720004")
#http://supercon.nims.go.jp/index_en.html
#https://github.com/vstanev1/Supercon

df = pd.DataFrame(dat)
path = "../../benchmarks/AI/SinglePropertyPrediction/mag2d_chem_magnetic_moment.json.zip"
js_tag = "mag2d_chem_magnetic_moment.json"
id_data = json.loads(zipfile.ZipFile(path).read(js_tag))
train_ids = np.array(list(id_data['train'].keys()),dtype='int')
test_ids = np.array(list(id_data['test'].keys()),dtype='int')
train_df = df[df['id'].isin(train_ids)]
test_df = df[df['id'].isin(test_ids)]

Loading the zipfile...
Loading completed.


In [31]:
df

,id,magnetic_moment,formula
0,49,5.0780,V1Cr1Ge2Te6
1,4,3.1560,Cr1Co1Si2Te6
2,212,3.0161,Ta1Cr1Sn1Ge1Te6
3,153,2.7459,Cr1Sn1Te6P1Au1
4,46,2.2800,Ti1Cr1Si2Te6
...,...,...,...
221,57,2.9670,Y1Cr1Si1Te6P1
222,97,5.6479,Cr1Ge2Te6W1
223,60,2.6981,Ta1Cr1In1Te6Pb1
224,152,5.0129,Cr1Sn1Te6Mo1P1


In [32]:
from jarvis.ai.descriptors.elemental import get_element_fraction_desc
from jarvis.ai.descriptors.cfid import CFID,get_chem_only_descriptors
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train=[]
y_train=[]
X_test=[]
y_test=[]
train_ids=[]
test_ids=[]
for i,ii in train_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_train.append(desc)
    y_train.append(ii['magnetic_moment'])
    train_ids.append(ii['id'])
for i,ii in test_df.iterrows():
    desc=get_element_fraction_desc(ii['formula'])
    #desc=get_chem_only_descriptors(ii['composition'])[0]
    X_test.append(desc)
    y_test.append(ii['magnetic_moment'])
    test_ids.append(ii['id'])
    
X_train=np.array(X_train,dtype='float')
y_train=np.array(y_train,dtype='float')
X_test=np.array(X_test,dtype='float')
y_test=np.array(y_test,dtype='float')
#Split train into train-val split
n_train = int(len(X_train)*0.8)
n_val = len(X_train)-n_train
n_train,n_val,X_train.shape
X_train = X_train[:n_train]
y_train = y_train[:n_train]
X_val = X_train[-n_val:]
y_val = y_train[-n_val:]

In [33]:
X_train.shape,X_test.shape

((144, 103), (46, 103))

In [34]:
import numpy as np
np.random.seed(1234567)
import tensorflow as tf
tf.random.set_seed(1234567)
import random
random.seed(1234567)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import mean_absolute_error, mean_squared_error
#Network from "Enabling deeper learning on big data for materials informatics applications"
#Link to paper: https://dl.acm.org/doi/10.1145/3292500.3330703 https://www.nature.com/articles/s41598-021-83193-1
in_layer = Input(shape=(X_train.shape[1],))

layer_1 = Dense(1024)(in_layer)
layer_1 = BatchNormalization()(layer_1)
layer_1 = Activation('relu')(layer_1)

fcc_1 = Dense(1024)(in_layer)
gsk_1 = add([fcc_1, layer_1])

layer_2 = Dense(1024)(gsk_1)
layer_2 = BatchNormalization()(layer_2)
layer_2 = Activation('relu')(layer_2)

gsk_2 = add([gsk_1, layer_2])

layer_3 = Dense(1024)(gsk_2)
layer_3 = BatchNormalization()(layer_3)
layer_3 = Activation('relu')(layer_3)

gsk_3 = add([gsk_2, layer_3])

layer_4 = Dense(1024)(gsk_3)
layer_4 = BatchNormalization()(layer_4)
layer_4 = Activation('relu')(layer_4)

gsk_4 = add([gsk_3, layer_4])

layer_5 = Dense(512)(gsk_4)
layer_5 = BatchNormalization()(layer_5)
layer_5 = Activation('relu')(layer_5)

mcc_5 = Dense(512)(gsk_4)
msk_5 = add([mcc_5, layer_5])

layer_6 = Dense(512)(msk_5)
layer_6 = BatchNormalization()(layer_6)
layer_6 = Activation('relu')(layer_6)

msk_6 = add([msk_5, layer_6])

layer_7 = Dense(512)(msk_6)
layer_7 = BatchNormalization()(layer_7)
layer_7 = Activation('relu')(layer_7)

msk_7 = add([msk_6, layer_7])

layer_8 = Dense(256)(msk_7)
layer_8 = BatchNormalization()(layer_8)
layer_8 = Activation('relu')(layer_8)

mcc_8 = Dense(256)(msk_7)
msk_8 = add([mcc_8, layer_8])

layer_9 = Dense(256)(msk_8)
layer_9 = BatchNormalization()(layer_9)
layer_9 = Activation('relu')(layer_9)

msk_9 = add([msk_8, layer_9])

layer_10 = Dense(256)(msk_9)
layer_10 = BatchNormalization()(layer_10)
layer_10 = Activation('relu')(layer_10)

msk_10 = add([msk_9, layer_10])

layer_11 = Dense(128)(layer_10)
layer_11 = BatchNormalization()(layer_11)
layer_11 = Activation('relu')(layer_11)

mcc_11 = Dense(128)(msk_10)
msk_11 = add([mcc_11, layer_11])

layer_12 = Dense(128)(msk_11)
layer_12 = BatchNormalization()(layer_12)
layer_12 = Activation('relu')(layer_12)

msk_12 = add([msk_11, layer_12])

layer_13 = Dense(128)(msk_12)
layer_13 = BatchNormalization()(layer_13)
layer_13 = Activation('relu')(layer_13)

msk_13 = add([msk_12, layer_13])

layer_14 = Dense(64)(msk_13)
layer_14 = BatchNormalization()(layer_14)
layer_14 = Activation('relu')(layer_14)

mcc_14 = Dense(64)(msk_13)
msk_14 = add([mcc_14, layer_14])

layer_15 = Dense(64)(msk_14)
layer_15 = BatchNormalization()(layer_15)
layer_15 = Activation('relu')(layer_15)

msk_15 = add([msk_14, layer_15])

layer_16 = Dense(32)(msk_15)
layer_16 = BatchNormalization()(layer_16)
layer_16 = Activation('relu')(layer_16)

mcc_16 = Dense(32)(msk_15)
msk_16 = add([mcc_16, layer_16])

out_layer = Dense(1)(msk_16)

model = Model(inputs=in_layer, outputs=out_layer)

adam = optimizers.Adam(lr=0.0001)
model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=adam, metrics=['mean_absolute_error'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100, restore_best_weights=True)


In [35]:
%%time
model.fit(X_train, y_train, verbose=2, validation_data=(X_val, y_val), epochs=3000, batch_size=32, callbacks=[es])

Epoch 1/3000
5/5 - 12s - loss: 27.9820 - mean_absolute_error: 27.9820 - val_loss: 2.6076 - val_mean_absolute_error: 2.6076 - 12s/epoch - 2s/step
Epoch 2/3000
5/5 - 0s - loss: 8.7095 - mean_absolute_error: 8.7095 - val_loss: 2.3022 - val_mean_absolute_error: 2.3022 - 210ms/epoch - 42ms/step
Epoch 3/3000
5/5 - 0s - loss: 4.9097 - mean_absolute_error: 4.9097 - val_loss: 2.1726 - val_mean_absolute_error: 2.1726 - 193ms/epoch - 39ms/step
Epoch 4/3000
5/5 - 0s - loss: 2.2379 - mean_absolute_error: 2.2379 - val_loss: 3.0155 - val_mean_absolute_error: 3.0155 - 178ms/epoch - 36ms/step
Epoch 5/3000
5/5 - 0s - loss: 2.0675 - mean_absolute_error: 2.0675 - val_loss: 3.1252 - val_mean_absolute_error: 3.1252 - 157ms/epoch - 31ms/step
Epoch 6/3000
5/5 - 0s - loss: 1.7692 - mean_absolute_error: 1.7692 - val_loss: 2.7939 - val_mean_absolute_error: 2.7939 - 169ms/epoch - 34ms/step
Epoch 7/3000
5/5 - 0s - loss: 1.5509 - mean_absolute_error: 1.5509 - val_loss: 3.0622 - val_mean_absolute_error: 3.0622 - 186

In [36]:
pred=model.predict(X_test)
print(mean_absolute_error(y_test,pred))

2/2 [==============================] - 0s 7ms/step
0.37738366399433293


In [37]:
import os
f=open('AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv','w')
f.write('id,target,prediction\n')
for i,j,k in zip(test_ids,y_test,pred):
    line=str(i)+','+str(j)+','+str(k[0])+'\n'
    f.write(line)
f.close()
cmd = 'zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv.zip AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)
cmd='rm AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv'
os.system(cmd)


  adding: AI-SinglePropertyPrediction-magnetic_moment-mag2d_chem-test-mae.csv (deflated 51%)


0